In [1]:
import os
os.environ['LANGCHAIN_PROJECT']="Microsoft Hackathon"

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("testFile.pdf")
pages = loader.load_and_split()

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size= 1000,
    chunk_overlap= 50 , 
    length_function= len ,
    is_separator_regex= False
)

In [4]:
texts = text_splitter.create_documents([pages[i].page_content for i in range(len(pages))])

In [5]:
from qdrant_client import QdrantClient

client = QdrantClient(url="http://localhost:6333")


In [6]:
from qdrant_client.models import Distance, VectorParams

client.create_collection(
    collection_name="MedicalPapers",
    vectors_config=VectorParams(size=1024, distance=Distance.DOT),
)


True

In [7]:
import cohere
import qdrant_client
from qdrant_client.models import Batch

cohere_client = cohere.Client("eFUHV96luEGkkcHOkuQlHu7gxocJUOwvE4jfLI9T")
client = qdrant_client.QdrantClient()
client.upsert(
    collection_name="MedicalPapers",
    points=Batch(
        ids=range(len(texts)),
        vectors=cohere_client.embed(
            model="embed-english-v3.0",  # New Embed v3 model
            input_type="search_document",  # Input type for indexing documents
            texts= [texts[i].page_content for i in range(len(texts))],
        ).embeddings,
        payloads = [{"Context{}".format(index): value} for index, value in enumerate([texts[i].page_content for i in range(len(texts))], start=1)],
    ),
)


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [8]:
results = client.search(
    collection_name="MedicalPapers",
    query_vector=cohere_client.embed(
        model="embed-english-v3.0",  
        input_type="search_query",  # Input type for search queries
        texts=["Breast Cancer"],
    ).embeddings[0],
)


In [9]:
len(results)

10

In [10]:
results[0]

ScoredPoint(id=1, version=0, score=0.46729892, payload={'Context2': 'generation. This eventually builds energetic stress and a decline in energy. Therefore, the\nnatural cellular processes and proliferating tumor cells are obstructed. Here, we used\nelectroporation, where, the MDA-MB-231, human TNBC cells were subjected to high intensity,\nshort-duration electrical pulses (EP) in the presence of Metformin. The cell viability results\nindicate lower cell viability of 43.45% as compared to 85.20% with drug alone at 5mM\nconcentration. This indicates that Metformin, the most common diabetes drug could also be\nexplored for cancer treatment.\nI. I NTRODUCTION\nTriple-Negative Breast Cancer (TNBC) is an aberrant type of breast cancer subset, with\nthe absence of traditional biomarkersi.e.,Estrogen and progesterone receptors, and shows\nno upregulation in HER2 protein. It is the most aggressive subset of all breast cancer types.\nHence, making the diagnosis and treatment of TNBC is indeed ch

In [11]:
from langchain.prompts import ChatPromptTemplate
promptTemplate = template = """You are a medical assistant that specializes in providing second opinions, diagnosing complex cases 
and suggesting treatment plans. When I describe the patient details, medical context and task, give me the appropriate treatment plan 
or second opinion based on the task given by analyzing the patient details and medical context. In your answer include how your opinion or treatment
plann is related to the patient's history.

Patient History : {patientHistory}

Medical Context : {context}

Task: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'patientHistory', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'patientHistory', 'question'], template="You are a medical assistant that specializes in providing second opinions, diagnosing complex cases \nand suggesting treatment plans. When I describe the patient details, medical context and task, give me the appropriate treatment plan \nor second opinion based on the task given by analyzing the patient details and medical context. In your answer include how your opinion or treatment\nplann is related to the patient's history.\n\nPatient History : {patientHistory}\n\nMedical Context : {context}\n\nTask: {question}\n"))])

In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-pro")

In [13]:
chain = prompt | llm 

In [14]:
patientHistory = """Patient Details : Patient Information:
Name: Rajesh Kumar
Age: 45
Gender: Male
Occupation: Office manager
Medical History:
Hypertension (controlled with medication)
Previous history of lumbar strain due to heavy lifting incidents, managed conservatively
Chief Complaint:
Chronic back pain
Duration: Approximately 6 months
Symptoms:
Dull, aching pain in the lower back, occasionally radiating down the right leg
Exacerbated by prolonged sitting or standing, relieved by lying down
Severity: 7/10 on average, worsening to 8/10 during flare-ups
No associated numbness or tingling
Impact on Daily Activities:
Difficulty performing work duties due to prolonged sitting at a desk
Limited ability to participate in recreational activities, such as playing sports or gardening
Struggles with household chores, particularly activities that involve bending or lifting
Treatment History:
Nonsteroidal anti-inflammatory drugs (NSAIDs) for pain management, with minimal relief
Physical therapy sessions focusing on core strengthening exercises and flexibility, providing short-term improvement but limited long-term benefits
Occasional use of heat packs and over-the-counter topical analgesics for symptomatic relief
Current Medications:
Amlodipine 5 mg daily for hypertension
Ibuprofen 400 mg as needed for back pain
Additional Concerns:
Rajesh is concerned about the impact of his back pain on his job performance and overall quality of life.
He's interested in exploring alternative treatment options, such as acupuncture or chiropractic care.
 """
context = """ Ibuprofen is used to relieve pain from various conditions such as headache, dental pain, menstrual cramps, muscle aches, or arthritis. It is also used to reduce fever and to relieve minor aches and pain due to the common cold or flu. Ibuprofen is a nonsteroidal anti-inflammatory drug (NSAID). It works by blocking your body's production of certain natural substances that cause inflammation. This effect helps to decrease swelling, pain, or fever.If you are treating a chronic condition such as arthritis, ask your doctor about non-drug treatments and/or using other medications to treat your pain. See also Warning section.This form of ibuprofen is intended for use in children. Do not give this medication to a child younger than 2 years unless directed by the doctor.Check the ingredients on the label even if you have used the product before. The manufacturer may have changed the ingredients. Also, products with similar names may contain different ingredients meant for different purposes. Taking the wrong product could harm you.
Amlodipine is used alone or in combination with other medications to treat high blood pressure in adults and children 6 years and older. It is also used to treat certain types of angina (chest pain) and coronary artery disease (narrowing of the blood vessels that supply blood to the heart) """

question = """ I'm facing challenges in managing a case of chronic back pain in a patient. Despite thorough evaluation and treatment attempts, including physical therapy and medication, the pain persists. The patient experiences a constant dull ache localized to the lumbar region, limiting their mobility and affecting their quality of life. I'm seeking your expertise to explore alternative treatment strategies or diagnostic approaches that may offer relief. """

In [1]:
patientHistory = """Patient Details : Patient Information:
Name: Rajesh Kumar
Age: 45
Gender: Male
Occupation: Office manager
Medical History:
Hypertension (controlled with medication)
Previous history of lumbar strain due to heavy lifting incidents, managed conservatively
Chief Complaint:
Chronic back pain
Duration: Approximately 6 months
Symptoms:
Dull, aching pain in the lower back, occasionally radiating down the right leg
Exacerbated by prolonged sitting or standing, relieved by lying down
Severity: 7/10 on average, worsening to 8/10 during flare-ups
No associated numbness or tingling
Impact on Daily Activities:
Difficulty performing work duties due to prolonged sitting at a desk
Limited ability to participate in recreational activities, such as playing sports or gardening
Struggles with household chores, particularly activities that involve bending or lifting
Treatment History:
Nonsteroidal anti-inflammatory drugs (NSAIDs) for pain management, with minimal relief
Physical therapy sessions focusing on core strengthening exercises and flexibility, providing short-term improvement but limited long-term benefits
Occasional use of heat packs and over-the-counter topical analgesics for symptomatic relief
Current Medications:
Amlodipine 5 mg daily for hypertension
Ibuprofen 400 mg as needed for back pain
Additional Concerns:
Rajesh is concerned about the impact of his back pain on his job performance and overall quality of life.
He's interested in exploring alternative treatment options, such as acupuncture or chiropractic care.
 """
context = """ Ibuprofen is used to relieve pain from various conditions such as headache, dental pain, menstrual cramps, muscle aches, or arthritis. It is also used to reduce fever and to relieve minor aches and pain due to the common cold or flu. Ibuprofen is a nonsteroidal anti-inflammatory drug (NSAID). It works by blocking your body's production of certain natural substances that cause inflammation. This effect helps to decrease swelling, pain, or fever.If you are treating a chronic condition such as arthritis, ask your doctor about non-drug treatments and/or using other medications to treat your pain. See also Warning section.This form of ibuprofen is intended for use in children. Do not give this medication to a child younger than 2 years unless directed by the doctor.Check the ingredients on the label even if you have used the product before. The manufacturer may have changed the ingredients. Also, products with similar names may contain different ingredients meant for different purposes. Taking the wrong product could harm you.
Amlodipine is used alone or in combination with other medications to treat high blood pressure in adults and children 6 years and older. It is also used to treat certain types of angina (chest pain) and coronary artery disease (narrowing of the blood vessels that supply blood to the heart) """

question = """ I'm facing challenges in managing a case of chronic back pain in a patient. Despite thorough evaluation and treatment attempts, including physical therapy and medication, the pain persists. The patient experiences a constant dull ache localized to the lumbar region, limiting their mobility and affecting their quality of life. I'm seeking your expertise to explore alternative treatment strategies or diagnostic approaches that may offer relief. """

In [2]:
result = chain.invoke({"context":context , "patientHistory":patientHistory,"question":question})

NameError: name 'chain' is not defined

In [16]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

import langchain
langchain.debug = True 

In [17]:
to_markdown(result.content)

> **Second Opinion and Treatment Plan:**
> 
> **Alternative Treatment Options:**
> 
> * **Acupuncture:** Acupuncture involves inserting thin needles into specific points on the body to stimulate the nervous system and release endorphins, which have pain-relieving effects.
> * **Chiropractic Care:** Chiropractors use spinal adjustments and manipulations to improve spinal alignment, reduce nerve compression, and alleviate pain.
> 
> **Diagnostic Approaches:**
> 
> * **Advanced Imaging:** An MRI or CT scan can provide detailed images of the lumbar spine to identify potential structural abnormalities, such as herniated discs or spinal stenosis.
> * **Electromyography (EMG):** This test measures the electrical activity of muscles and nerves to assess nerve function and identify any nerve damage.
> 
> **Treatment Plan:**
> 
> Based on the patient's history and persistent pain despite conservative treatment, the following treatment plan is recommended:
> 
> * **Refer for Advanced Imaging:** An MRI or CT scan should be performed to rule out any underlying structural abnormalities contributing to the pain.
> * **Consider Epidural Steroid Injection:** If advanced imaging reveals a herniated disc or other nerve compression, an epidural steroid injection can be administered to reduce inflammation and provide pain relief.
> * **Explore Physical Therapy with a Focus on Pain Management:** Physical therapy can include techniques such as massage, ultrasound therapy, and electrotherapy to manage pain and improve mobility.
> * **Recommend Alternative Therapies:** Acupuncture or chiropractic care may be considered as complementary therapies to alleviate pain and improve function.
> * **Monitor Medication Use:** Ibuprofen is an NSAID and should be used cautiously due to its potential side effects, such as gastrointestinal upset and increased risk of cardiovascular events. The patient should be advised to use it as directed and only when necessary.
> * **Consider Alternative Medications:** If NSAIDs provide inadequate pain relief, other medications such as opioids or muscle relaxants may be considered. However, these should be used with caution and under close medical supervision.
> 
> **Relation to Patient's History:**
> 
> This treatment plan considers the patient's history of lumbar strain, which may have contributed to the development of chronic back pain. The recommended diagnostic tests and therapies aim to address potential underlying structural abnormalities or nerve damage that may be exacerbating the pain.

In [19]:
llm2 = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")
chain2 = prompt|llm2

In [20]:
result = chain2.invoke({"context":context , "patientHistory":patientHistory,"question":question})
to_markdown(result)

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "context": " Ibuprofen is used to relieve pain from various conditions such as headache, dental pain, menstrual cramps, muscle aches, or arthritis. It is also used to reduce fever and to relieve minor aches and pain due to the common cold or flu. Ibuprofen is a nonsteroidal anti-inflammatory drug (NSAID). It works by blocking your body's production of certain natural substances that cause inflammation. This effect helps to decrease swelling, pain, or fever.If you are treating a chronic condition such as arthritis, ask your doctor about non-drug treatments and/or using other medications to treat your pain. See also Warning section.This form of ibuprofen is intended for use in children. Do not give this medication to a child younger than 2 years unless directed by the doctor.Check the ingredients on the label even if you have used the product before. The manufacturer may have changed the ingredients. Also, produc

AttributeError: 'AIMessage' object has no attribute 'replace'

In [ ]:
to_markdown(result.content)

In [ ]:
#Query Construction
def generate_queries(model,prompt, num_queries):
  query_generation_prompt = ChatPromptTemplate.from_template("Given the prompt: '{prompt}', generate {num_queries} questions that are better articulated. Return in the form of an list. For example: ['question 1', 'question 2', 'question 3']")
  query_generation_chain = query_generation_prompt | model
  return str_to_json(query_generation_chain.invoke({"prompt": prompt, "num_queries": num_queries}).content)

In [ ]:
def evaluate_with_llm(model, prompt, generated_text):
    evaluations = {}

    # Template for creating evaluation queries
    def create_evaluation_query(template, **kwargs):
        query = ChatPromptTemplate.from_template(template)
        chain = query | model
        return float(chain.invoke(kwargs).content)

    # Evaluate Relevance
    relevance_template = "Given the context provided by the following prompt: '{prompt}', please evaluate on a scale from 0 to 1, where 1 is highly relevant and 0 is not relevant at all, how relevant is this generated response: '{generated_text}'? Provide a numerical score only."
    evaluations['relevance'] = create_evaluation_query(relevance_template, prompt=prompt, generated_text=generated_text)

    # Evaluate Clarity
    clarity_template = "How clear and easily understandable is this text: '{generated_text}'? Rate its clarity on a scale from 0 to 1, where 1 indicates that the text is very clear and 0 indicates that the text is very unclear. Provide a numerical score only."
    evaluations['clarity'] = create_evaluation_query(clarity_template, prompt=prompt, generated_text=generated_text)

    # Evaluate Coherence
    coherence_template = "On a scale from 0 to 1, with 1 being highly coherent and 0 being not coherent at all, how well do the ideas in this generated text: '{generated_text}' flow together? Consider if the text makes logical sense as a whole. Provide a numerical score only."
    evaluations['coherence'] = create_evaluation_query(coherence_template, prompt=prompt, generated_text=generated_text)

    # Evaluate Detail and Exhaustiveness
    detail_template = "Assessing the detail and exhaustiveness relative to the prompt '{prompt}', how thoroughly does this generated text: '{generated_text}' cover the topic? Rate on a scale from 0 to 1, where 1 is very detailed and exhaustive, and 0 is not detailed at all. Provide a numerical score only."
    evaluations['details'] = create_evaluation_query(detail_template, prompt=prompt, generated_text=generated_text)

    # Evaluate Suitability as an Answer
    suitability_template = "Evaluate the suitability of this generated text: '{generated_text}' as an answer to the original prompt '{prompt}'. On a scale from 0 to 1, where 1 is a perfect answer and 0 is completely unsuitable, provide a numerical score only."
    evaluations['suitability'] = create_evaluation_query(suitability_template, prompt=prompt, generated_text=generated_text)

    return evaluations

In [ ]:
def critique(model, prompt, generated_text):
    evaluation_weights = {
        'relevance': 3,  
        'clarity': 1,
        'coherence': 0.5,
        'details': 1.5,
        'suitability': 2  
    }
    
    evaluations = evaluate_with_llm(model, prompt, generated_text)
    print("Evaluations:", evaluations)
    
    # Calculate the weighted sum of the evaluations
    weighted_sum = sum(evaluations[aspect] * evaluation_weights.get(aspect, 1) for aspect in evaluations)
    
    # Calculate the sum of weights for the aspects evaluated
    total_weight = sum(evaluation_weights.get(aspect, 1) for aspect in evaluations)
    
    # Calculate the weighted average of the evaluations
    weighted_average = weighted_sum / total_weight if total_weight > 0 else 0
    
    return [weighted_average, evaluations]

In [ ]:
import cohere
from langchain_community.utilities import SerpAPIWrapper
co = cohere.Client(os.environ["COHERE_API_KEY"])


def get_reranked_result(query, top_n=1):
  matches = kb.query([Query(text=query)])
  docs = extract_documents_texts(matches)
  rerank_results = co.rerank(model="rerank-english-v2.0", query=query, documents=docs, top_n=top_n)
  texts = []
  for rerank_result in rerank_results:
      # Accessing the 'text' field in the document attribute of each RerankResult
      text = rerank_result.document['text']
      texts.append(text)
  return texts

In [ ]:
from typing import List, Dict, Any, Tuple
from collections import defaultdict

class QueryDetail:
    def __init__(self, query: str):
        self.query = query
        self.content: List[str] = []
        self.critique_score: float = 0.0
        self.critique_details: Dict[str, Any] = {}
        self.retrieval_needed: bool = False
        self.search_needed: bool = False

    def add_response(self, model, search) -> None:
        """Process the query to add response, handle retrieval and critique."""
        if is_retrieval_needed(model, self.query):
            response = " ".join(get_reranked_result(self.query, top_n=3))
            self.retrieval_needed = True
        else:
            response = "Some generated answer"
            self.retrieval_needed = False
        
        self.content.append(response)
        
        critique_score, critique_details = critique(model, self.query, response)
        self.critique_score = critique_score
        self.critique_details = critique_details
        self.search_needed = critique_score < 0.5

        if self.search_needed:
            self.search_and_add_results(search)

    def search_and_add_results(self, search) -> None:
        """Perform a search and process the results if critique score is low."""
        search_result_raw = search.run(self.query)
        search_result = str_to_json(search_result_raw) or []
        self.content.extend(search_result)

class QueryProcessor:
    def __init__(self, model, search, queries: List[str]):
        self.model = model
        self.search = search
        self.queries = [QueryDetail(query) for query in queries]

    def process_queries(self) -> List[QueryDetail]:
        """Process each query in the list."""
        for query_detail in self.queries:
            query_detail.add_response(self.model, self.search)
            if query_detail.search_needed:
                consolidated_response = consolidate(self.model, query_detail.content)
                query_detail.content = [consolidated_response]
                critique_score, critique_details = critique(self.model, query_detail.query, consolidated_response)
                query_detail.critique_score = critique_score
                query_detail.critique_details = critique_details
        return self.queries

def advanced_rag_query(model, query: str, num_queries: int) -> List[QueryDetail]:
    search = SerpAPIWrapper()
    initial_queries = generate_queries(model, query, num_queries)[:num_queries]
    query_processor = QueryProcessor(model, search, initial_queries)
    processed_queries = query_processor.process_queries()
    return processed_queries